In [1]:
import findspark

findspark.init()
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, BooleanType, ArrayType, MapType
from pyspark.sql.functions import col, struct, when, lit, sum, expr, array_contains, udf, upper, explode, row_number, rank, dense_rank, lead, current_date, date_format, to_date, datediff, from_json, to_json, json_tuple, get_json_object, collect_set, count, concat_ws, concat, monotonically_increasing_id, format_string, length
from pyspark.sql.window import Window
import json
import time

In [ ]:
my_spark = SparkSession.builder.appName('comparision_v1').getOrCreate()

In [ ]:
df_a = my_spark.read.csv('./a.csv', header=True)
df_b = my_spark.read.csv('./b.csv', header=True)
WindowSpec = Window.orderBy('name', 'enroll')
# df_a = df_a.withColumn('rn', row_number().over(WindowSpec))
df_a = df_a.withColumn('rn', monotonically_increasing_id())
# df_b = df_b.withColumn('rn', row_number().over(WindowSpec))
df_b = df_b.withColumn('rn', monotonically_increasing_id())
df_a = df_a.select([col(col_name).alias('a_'+col_name) for col_name in df_a.columns])
df_b = df_b.select([col(col_name).alias('b_'+col_name) for col_name in df_b.columns])
df_a_b = df_a.join(df_b, on=(df_a['a_rn'] == df_b['b_rn']), how='inner')
df_a_b.show()

In [ ]:
lst_cols = list(zip(df_a.columns, df_b.columns))
for ct in lst_cols:
    df_a_b = df_a_b.withColumn(
        f'cond_{ct[0]}_{ct[1]}',
        when(
            (col(ct[0]).isNull()),
            concat(lit('Null Data -> Columns: '), lit(ct[0]), lit(' <> '),
                   lit(ct[1]), lit(', Values: '), lit('null'), lit(' <> '),
                   col(ct[1]), lit(', id: '), col('a_rn'))).when(
                       (length(col(ct[1])) > length(col(ct[0]))) &
                       (col(ct[1]).contains(col(ct[0]))),
                       concat(lit('Precision Data -> Columns: '), lit(ct[0]),
                              lit(' <> '), lit(ct[1]), lit(', Values: '),
                              lit('null'), lit(' <> '), col(ct[1]),
                              lit(', id: '), col('a_rn'))).when(
                                  (col(ct[0]) != col(ct[1])),
                                  concat(lit('Unmatched Data -> Columns: '),
                                         lit(ct[0]), lit(' <> '), lit(ct[1]),
                                         lit(', Values: '), col(ct[0]),
                                         lit(' <> '), col(ct[1]),
                                         lit(', id: '),
                                         col('a_rn'))).otherwise(f'matched_e'))
df_a_b = df_a_b.drop(*[i for i in df_a.columns + df_b.columns])
df_a_b = df_a_b.toPandas()

In [ ]:
res = [j for col in df_a_b.columns for j in df_a_b[col] if j != 'matched_e']
print('\n'.join(res))

In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession, Row
from pyspark.sql.types import StructField, StructType, StringType, IntegerType, BooleanType, ArrayType, MapType
from pyspark.sql.functions import col, struct, when, lit, sum, expr, array_contains, udf, upper, explode, row_number, rank, dense_rank, lead, current_date, date_format, to_date, datediff, from_json, to_json, json_tuple, get_json_object, collect_set
from pyspark.sql.window import Window
from pyspark.sql.types import StringType
import json
import time
spark = SparkSession.builder.master('local[1]').appName('ali_spark_cond').getOrCreate()
# spark.udf.registerJavaFunction('encrypt_sal', 'bt.encryptsal.EncryptSalary', StringType())

In [9]:
data2 = [("James","","Smith","36636","M",3000),
    ("Michael","Rose","","40288","M",4000),
    ("Robert","","Williams","42114","M",4000),
    ("Maria","Anne","Jones","39192","F",4000),
    ("Jen","Mary","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("firstname",StringType(),True), \
    StructField("middlename",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=data2,schema=schema).cache()
# df.printSchema()
df = df.filter(df['gender'] == 'M')
df.write.csv("e3.csv")
# for (id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
#     rdd.unpersist()

In [11]:
for (id, rdd) in spark.sparkContext._jsc.getPersistentRDDs().items():
    print(id, rdd)

24 Scan ExistingRDD[firstname#156,middlename#157,lastname#158,id#159,gender#160,salary#161]
 MapPartitionsRDD[24] at csv at NativeMethodAccessorImpl.java:0
